# Image Translation: Image to Sketch using U-NET Architecture
We will be using U-NET architecture to translate facial picture to its sketch using CUHK facial dataset.\


![Title Image](https://www.researchgate.net/profile/Vinayakumar_Ravi/publication/347776755/figure/download/fig1/AS:976145776377858@1609742697583/Convolutional-Neural-Network-architecture.png)\
Note: This image has nothing to do with the model being trained :)
Base Reference: https://www.kaggle.com/code/theblackmamba31/photo-to-sketch-using-autoencoder

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.utils import img_to_array
import keras.applications.resnet as resnet
import keras.backend as K
import tqdm
import cv2

## GPU Connectivity
To speed up the training, we will be using GPU. While this code is unnecessary in kaggle, but this code can be helpful when using your offline gpu.\
Make sure to select the accelator as one of the gpus, preferably GPU P100 while running on kaggle.

In [ ]:
# Configure TensorFlow to use GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        # Restrict TensorFlow to only use the first GPU
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

In [ ]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Importing Dataset
Here, we are importing images, and storing them collectively.

In [ ]:
import os
image_path = '/kaggle/input/cuhk-face-sketch-database-cufs/photos' #'data/photos'
sketches_path = '/kaggle/input/cuhk-face-sketch-database-cufs/sketches' #'data/sketches'

image_names = []
sketch_names = []

for i in os.listdir(image_path):
    image_names.append(i)

for i in os.listdir(sketches_path):
    sketch_names.append(i)
image_names = sorted(image_names)
sketch_names = sorted(sketch_names)
print("Images Names", image_names)
print("Sketch Names", sketch_names)

# Data Augmentation
Data augmentation is a very important step. As our dataset is pretty small, we will augment it to create new images by flipping and rotating them.\
This will give us around 1504 images for training and testing.\
Size is kept to 256, if working on offline GPU, and GPU memory is not greater, suggested to lower the size to 128 or around 100.

In [ ]:
img_array = []
sketch_array = []
size = 256

for img_name in tqdm.tqdm(image_names):
    img = image_path +  "/" + img_name
    img = cv2.imread(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (size, size))
    img = img.astype('float32')/255
    # Appending images to img_array
    img_array.append(img_to_array(img))

    ## Augmentation
    # Horizontal Flip on Upright Image
    img1 = cv2.flip(img, 1)
    # Vertical Flip to Downright Image
    img2 = cv2.flip(img, -1)
    # Horizontal Flip on Downright Image
    img3 = cv2.flip(img2, 1)
    # Clockwise Rotate 90 degrees
    img4 = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    # Clockwise Rotate 270 degrees
    img5 = cv2.flip(img4, 1)
    # Counter Clockwise Rotate 90 degrees
    img6 = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
    # Counter Clockwise Rotate 270 degrees
    img7 = cv2.flip(img, 1)
    img_array.append(img_to_array(img1))
    img_array.append(img_to_array(img2))
    img_array.append(img_to_array(img3))
    img_array.append(img_to_array(img4))
    img_array.append(img_to_array(img5))
    img_array.append(img_to_array(img6))
    img_array.append(img_to_array(img7))

for img_name in tqdm.tqdm(sketch_names):
    img = sketches_path +  "/" + img_name
    img = cv2.imread(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (size, size))
    img = img.astype('float32')/255
    # Appending images to img_array
    sketch_array.append(img_to_array(img))

    ## Augmentation
    # Horizontal Flip on Upright Image
    img1 = cv2.flip(img, 1)
    # Vertical Flip to Downright Image
    img2 = cv2.flip(img, -1)
    # Horizontal Flip on Downright Image
    img3 = cv2.flip(img2, 1)
    # Clockwise Rotate 90 degrees
    img4 = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    # Clockwise Rotate 270 degrees
    img5 = cv2.flip(img4, 1)
    # Counter Clockwise Rotate 90 degrees
    img6 = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
    # Counter Clockwise Rotate 270 degrees
    img7 = cv2.flip(img, 1)
    sketch_array.append(img_to_array(img1))
    sketch_array.append(img_to_array(img2))
    sketch_array.append(img_to_array(img3))
    sketch_array.append(img_to_array(img4))
    sketch_array.append(img_to_array(img5))
    sketch_array.append(img_to_array(img6))
    sketch_array.append(img_to_array(img7))

In [ ]:
print(f"The number of total Images is: {len(sketch_array)}")

## Splitting the Data: Train & Valid
Here, we will split our dataset into training and validation i.e. testing datasets.

In [ ]:
train_sketch_images = sketch_array[:1400]
train_original_images = img_array[:1400]
test_sketch_images = sketch_array[1400:]
test_original_images = img_array[1400:]
train_sketch_images = np.reshape(train_sketch_images, (len(train_sketch_images), int(size), int(size), 3))
train_original_images = np.reshape(train_original_images, (len(train_original_images), int(size), int(size), 3))
print('Train sketch images size: ', train_sketch_images.shape)
test_sketch_images = np.reshape(test_sketch_images, (len(test_sketch_images), int(size), int(size), 3))
test_original_images = np.reshape(test_original_images, (len(test_original_images), int(size), int(size), 3))
print('Test sketch images size: ', test_sketch_images.shape)

## Modelling: Defining Model Structure
The most important step in our notebook / tutorial. Here we are defining the structure of our model which resmbles or is imitated as a U-NET architecture.

In [ ]:
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate
from keras.models import Model

# Define the U-Net architecture
def unet_model(input_shape):
    # Input Layer
    inputs = Input(input_shape)

    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)

    # Decoder
    up6 = Conv2DTranspose(512, 2, strides=(2, 2), padding='same')(conv5)
    up6 = concatenate([up6, conv4])
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(up6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = Conv2DTranspose(256, 2, strides=(2, 2), padding='same')(conv6)
    up7 = concatenate([up7, conv3])
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(up7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(conv7)
    up8 = concatenate([up8, conv2])
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(up8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv8)
    up9 = concatenate([up9, conv1])
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(up9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)

    # Output layer
    output = Conv2D(3, 1, activation='linear', padding='same')(conv9)

    # Create the model
    model = Model(inputs=inputs, outputs=output)

    return model

# Define the input shape
input_shape = (int(size), int(size), 3)

# Create the U-Net model
model = unet_model(input_shape)

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Print the model summary
final_model = model
final_model.summary()

## Modelling: Training the model
We are going to use 'val_loss' i.e. validation loss to incorporate early stopping if possible, it is an important step if you don't want to waste time upon very minor or no growth in precision of the model.\
You can introduce batch_size parameter if you are facing memory exceeding errors, keep it as small as possible, but keep it as an integer.

In [ ]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
final_model.fit(train_original_images, train_sketch_images, epochs=100,
                validation_data=(test_original_images, test_sketch_images),
                verbose=1, callbacks=[early_stopping])

## Saving the model

In [ ]:
# Saving the model
final_model.save_weights('unet_weights_kaggle')
final_model.save('unet_sketch_gen_kaggle.h5')

## Testing the Predictions

In [ ]:
def show_images(real, sketch, predicted):
    plt.figure(figsize=(12, 12))
    plt.subplot(1, 3, 1)
    plt.title("Image", fontsize=15, color='Lime')
    plt.imshow(real)
    plt.subplot(1, 3, 2)
    plt.title("sketch", fontsize=15, color='Blue')
    plt.imshow(sketch)
    plt.subplot(1, 3, 3)
    plt.title("Predicted", fontsize=15, color='gold')
    plt.imshow(predicted)


ls = [i for i in range(0, 10, 8)]
for i in ls:
    predicted = np.clip(final_model.predict(test_original_images[i].reshape(1, size, size, 3)), 0.0, 1.0).reshape(size, size,                                                               3)
    show_images(test_original_images[i], test_sketch_images[i], predicted)

## Evaluation

In [ ]:
prediction_on_test_data = final_model.evaluate(test_original_images, test_sketch_images)
print("Loss: ", prediction_on_test_data)
print("Accuracy: ", 100 - np.round(prediction_on_test_data * 100,3))